# Data Warehouse Truven QA - Claim Header

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn()+' keepalives=1 keepalives_idle=30 keepalives_interval=10')
connection.autocommit = True

In [3]:
year_df = pd.read_sql('select distinct year from dw_staging.trum_claim_header;', con=connection)
tables = ['ccaes', 'mdcrs', 'mdcro', 'ccaeo']

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## Row Count and Claim Count

Similar to the member_enrollment_monthly table, the claim_header table has the row count should equal the number of unique claims.

For this table, we extract claim data from the s, o, and f tables from the truven schema. 


In [4]:
query = ''' drop table if exists qa_reporting.dw_truv_claim_header_counts;
create table qa_reporting.dw_truv_claim_header_counts
(
    data_source bpchar(4),
    calendar_year int,
    table_src text,
    dw_row_count int,
    src_row_count int,
    row_count_diff int,
    row_count_diff_percentage float,
    dw_uth_clm_id_count int,
    dw_src_clm_id_count int,
    src_clm_count int,
    clm_count_diff int,
    clm_count_percentage float,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [5]:
with connection.cursor() as cursor:
    for year in tqdm(year_df['year']):
        query = f'''
        insert into qa_reporting.dw_truv_claim_header_counts
        (data_source, calendar_year, table_src, dw_row_count, date_generated)
        select data_source, year, table_id_src, count(*), current_date
        from dw_staging.truc_claim_header
        where year = {year}
        group by 1, 2, 3
        '''
        
        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_uth_clm_id_count = count
        from (
            select data_source, year, table_id_src,  count(distinct uth_claim_id) as count 
            from dw_staging.truc_claim_header
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_src_clm_id_count = count
        from (
            select data_source, year, table_id_src,  count(distinct claim_id_src) as count 
            from dw_staging.truc_claim_header
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_uth_mbr_id_count = count
        from (
            select data_source, year, table_id_src, count(distinct uth_member_id) as count 
            from dw_staging.truc_claim_header
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)
        
        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_src_mbr_id_count = count
        from (
            select data_source, year, table_id_src, count(distinct member_id_src) as count 
            from dw_staging.truc_claim_header
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

100%|██████████| 12/12 [39:48<00:00, 199.08s/it]


In [6]:
with connection.cursor() as cursor:
    for year in tqdm(year_df['year']):
        query = f'''
        insert into qa_reporting.dw_truv_claim_header_counts
        (data_source, calendar_year, table_src, dw_row_count, date_generated)
        select data_source, year, table_id_src, count(*), current_date
        from dw_staging.trum_claim_header
        where year = {year}
        group by 1, 2, 3
        '''
        
        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_uth_clm_id_count = count
        from (
            select data_source, year, table_id_src, count(distinct uth_claim_id) as count 
            from dw_staging.trum_claim_header
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_src_clm_id_count = count
        from (
            select data_source, year, table_id_src, count(distinct claim_id_src) as count 
            from dw_staging.trum_claim_header
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_uth_mbr_id_count = count
        from (
            select data_source, year, table_id_src, count(distinct uth_member_id) as count 
            from dw_staging.trum_claim_header
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)
        
        query = f'''
        update qa_reporting.dw_truv_claim_header_counts b
        set dw_src_mbr_id_count = count
        from (
            select data_source, year, table_id_src, count(distinct member_id_src) as count 
            from dw_staging.trum_claim_header
        where year = {year}
        group by 1, 2, 3) a
        where a.year = b.calendar_year
        and a.table_id_src = b.table_src
        '''

        cursor.execute(query)

100%|██████████| 12/12 [07:40<00:00, 38.39s/it]


In [9]:
with connection.cursor() as cursor:
    query = '''update qa_reporting.dw_truv_claim_header_counts a
    set src_row_count = b.row_count,
    row_count_diff = dw_row_count - b.row_count,
    row_count_diff_percentage = 100. * abs( dw_row_count - b.row_count) / b.row_count,
    src_clm_count = clm_count,
    clm_count_diff = dw_uth_clm_id_count - b.clm_count,
    clm_count_percentage = 100. * abs(dw_uth_clm_id_count - b.clm_count) / b.clm_count,
    src_mbr_count = b.pat_count,
    mbr_count_diff = dw_uth_mbr_id_count - b.pat_count,
    mbr_count_percentage = 100. * abs(dw_uth_mbr_id_count - b.pat_count) / b.pat_count
    from qa_reporting.truven_counts b
    where a.calendar_year = b.year
    and a.table_src = b.table_name
    ;
    '''

    cursor.execute(query)

Note that the source row count from the o and s tables from truven are at detail level. This means that the row count from the individual tables are going to larger than the row count in the claim header table. Instead of comparing the row count from the DW table with the raw table, we can compare the row count with the unique claim count from the raw table.

In [3]:
df = pd.read_sql('select * from qa_reporting.dw_truv_claim_header_counts;', con=connection)
df.head()

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
0,truc,2013,ccaeo,374336664,875696525,-501359861,57.252695,374336664,374332695,374452293,-115629,0.030880,33649089,33649089,33659811,-10722,0.031854,2023-07-27
1,truc,2013,ccaes,16809774,62632638,-45822864,73.161319,16809774,16805702,16811236,-1462,0.008697,1720254,1720254,1720897,-643,0.037364,2023-07-27
2,truc,2019,ccaeo,242607491,566058850,-323451359,57.140942,242607491,242604185,242648619,-41128,0.016950,20277528,20277528,20280269,-2741,0.013516,2023-07-27
3,truc,2019,ccaes,10314198,37047879,-26733681,72.159815,10314198,10310236,10313175,1023,0.009919,941884,941884,942328,-444,0.047117,2023-07-27
4,truc,2022,ccaeo,165496616,371849172,-206352556,55.493617,165496616,165496514,165571401,-74785,0.045168,16045204,16045204,16057626,-12422,0.077359,2023-07-27


In [4]:
df[(df['data_source'] == 'trum')].sort_values(['calendar_year', 'table_src'])

,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
38,trum,2011,mdcro,106545093,234255047,-127709954,54.517482,106545093,106542800,106577308,-32215,0.030227,4743651,4743651,4750926,-7275,0.153128,2023-07-27
40,trum,2011,mdcrs,10833438,32535953,-21702515,66.703179,10833438,10831969,10836750,-3312,0.030563,794174,794174,794571,-397,0.049964,2023-07-27
39,trum,2012,mdcro,99412248,229768218,-130355970,56.733682,99412248,99410354,99450749,-38501,0.038714,4384481,4384481,4391031,-6550,0.149168,2023-07-27
41,trum,2012,mdcrs,10013746,32200251,-22186505,68.901652,10013746,10011723,10014407,-661,0.006600,714024,714024,714226,-202,0.028282,2023-07-27
28,trum,2013,mdcro,88011596,208095580,-120083984,57.706168,88011596,88009897,88058549,-46953,0.053320,3805817,3805817,3812531,-6714,0.176103,2023-07-27
29,trum,2013,mdcrs,8710381,29045752,-20335371,70.011515,8710381,8708514,8711803,-1422,0.016323,599538,599538,599773,-235,0.039181,2023-07-27
42,trum,2014,mdcro,79484204,193877526,-114393322,59.002879,79484204,79481580,79544615,-60411,0.075946,3505530,3505530,3514364,-8834,0.251368,2023-07-27
43,trum,2014,mdcrs,7963924,26044601,-18080677,69.421977,7963924,7961307,7964806,-882,0.011074,531316,531316,531556,-240,0.045150,2023-07-27
44,trum,2015,mdcro,49709576,121473554,-71763978,59.077862,49709576,49708196,49736837,-27261,0.054810,2032351,2032351,2034639,-2288,0.112452,2023-07-27
45,trum,2015,mdcrs,5185230,16705568,-11520338,68.961067,5185230,5183316,5184315,915,0.017649,336223,336223,336283,-60,0.017842,2023-07-27


In [13]:
df['row_count_diff_percentage'].describe()

count    48.000000
mean     63.585781
std       6.684626
min      54.517482
25%      57.527988
50%      62.711678
75%      70.031155
max      73.389903
Name: row_count_diff_percentage, dtype: float64

In [14]:
df['clm_count_percentage'].describe()

count    48.000000
mean      0.038273
std       0.055683
min       0.001538
25%       0.011345
50%       0.029503
75%       0.046599
max       0.383738
Name: clm_count_percentage, dtype: float64

In [15]:
df['mbr_count_percentage'].describe()

count    48.000000
mean      0.084027
std       0.126436
min       0.008867
25%       0.027830
50%       0.050361
75%       0.090472
max       0.856400
Name: mbr_count_percentage, dtype: float64

## Claim Type

In [16]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.truv_claim_type_count;
    with truv_claims as (
        select 'mdcro' as table_src, year, claim_id_derv, facprof
        from truven.mdcro
        union
        select 'mdcrs' as table_src, year, claim_id_derv, facprof
        from truven.mdcrs
    )
select table_src, year, facprof, count(*)
into qa_reporting.truv_claim_type_count
from truv_claims
group by 1,2,3
    '''
    cursor.execute(query)


    query = '''
    with truv_claims as (
        select 'ccaeo' as table_src, year, claim_id_derv, facprof
        from truven.ccaeo
        union
        select 'ccaes' as table_src, year, claim_id_derv, facprof
        from truven.ccaes
    )
    insert into qa_reporting.truv_claim_type_count
select table_src, year, facprof, count(*)
from truv_claims
group by 1,2,3
    '''
    cursor.execute(query)


In [18]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.dw_truv_claim_type_count;
select table_id_src, year, claim_type, count(*)
into qa_reporting.dw_truv_claim_type_count
from dw_staging.trum_claim_header
group by 1,2,3
    '''
    cursor.execute(query)

    query = '''
insert into qa_reporting.dw_truv_claim_type_count
select table_id_src, year, claim_type, count(*)
from dw_staging.truc_claim_header
group by 1,2,3
    '''
    cursor.execute(query)

In [19]:
query = f'''
select a.table_id_src, a.year, a.claim_type, a.count as dw_claim_type_count, b.count as src_claim_type_count, 
        a.count-b.count as claim_type_count_difference, 
         100. * abs(a.count-b.count) / b.count as claim_type_count_difference_percentage
from qa_reporting.dw_truv_claim_type_count a
full outer join qa_reporting.truv_claim_type_count b
on a.year =b.year
and a.claim_type = b.facprof
and a.table_id_src = b.table_src
'''

pd.read_sql(query, con=connection).sort_values(['year', 'claim_type'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,table_id_src,year,claim_type,dw_claim_type_count,src_claim_type_count,claim_type_count_difference,claim_type_count_difference_percentage
3,ccaeo,2011,F,55727659,55741449,-13790,0.024739
53,mdcro,2011,F,13497693,13501590,-3897,0.028863
59,mdcrs,2011,F,1249943,1250249,-306,0.024475
86,ccaes,2011,F,3745968,3746955,-987,0.026341
1,ccaeo,2011,P,409891153,409980920,-89767,0.021895
...,...,...,...,...,...,...,...
94,mdcrs,2022,F,257379,257435,-56,0.021753
11,mdcro,2022,P,25490483,25498036,-7553,0.029622
30,ccaes,2022,P,5183637,5202080,-18443,0.354531
43,ccaeo,2022,P,148608165,148667475,-59310,0.039894


In [20]:
pd.read_sql(query, con=connection).sort_values(['claim_type_count_difference_percentage'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,table_id_src,year,claim_type,dw_claim_type_count,src_claim_type_count,claim_type_count_difference,claim_type_count_difference_percentage
9,ccaes,2019,F,1664411,1664398,13,0.000781
87,mdcrs,2021,P,3330474,3330440,34,0.001021
76,mdcrs,2016,P,4284423,4284519,-96,0.002241
77,mdcrs,2019,P,4009890,4009787,103,0.002569
13,mdcrs,2017,P,3094559,3094470,89,0.002876
...,...,...,...,...,...,...,...
3,mdcro,2014,F,10992504,11003668,-11164,0.101457
23,ccaeo,2017,F,28361533,28391983,-30450,0.107249
58,mdcro,2017,F,5081956,5088331,-6375,0.125287
0,ccaes,2022,P,5183637,5202080,-18443,0.354531


## Costs

### Sum

In [21]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.truv_costs_total;

    create table qa_reporting.truv_costs_total
    (
        year int,
        table_name text,
        total_netpay bigint,
        total_pay bigint
    );
    '''
    cursor.execute(query)

In [22]:
for table in tables:
    print(table)
    with connection.cursor() as cursor:
        query = f'''insert into qa_reporting.truv_costs_total
    select year, '{table}' as table_name, sum(netpay) total_netpay, sum(pay) total_pay
    from truven.{table}
    group by 1
        '''
        cursor.execute(query)

ccaes
mdcrs
mdcro
ccaeo


In [23]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.dw_truv_costs_total;
select year, table_id_src,
        sum(total_charge_amount) as total_charge_amount,
        sum(total_allowed_amount) as total_allowed_amount, 
        sum(total_paid_amount) as total_paid_amount
into qa_reporting.dw_truv_costs_total
from dw_staging.trum_claim_header
group by 1,2
    '''
    cursor.execute(query)

    query = '''
insert into qa_reporting.dw_truv_costs_total
select year, table_id_src,
        sum(total_charge_amount) as total_charge_amount,
        sum(total_allowed_amount) as total_allowed_amount, 
        sum(total_paid_amount) as total_paid_amount
from dw_staging.truc_claim_header
group by 1,2
    '''
    cursor.execute(query)

In [24]:
query = f'''
select a.year, b.table_name, 
        a.total_paid_amount as dw_total_paid, b.total_netpay as src_total_paid, 
        a.total_paid_amount-b.total_netpay as total_paid_difference, 
        100. * abs(a.total_paid_amount-b.total_netpay) / b.total_netpay as total_paid_difference_percentage,
        a.total_allowed_amount as dw_total_allowed, b.total_pay as src_total_allowed,
        a.total_allowed_amount-b.total_pay as total_allowed_difference, 
        100. * abs(a.total_allowed_amount-b.total_pay) / b.total_pay as total_allowed_difference_percentage
from qa_reporting.dw_truv_costs_total a
full outer join qa_reporting.truv_costs_total b
on a.year =b.year
and a.table_id_src = b.table_name
'''

pd.read_sql(query, con=connection).sort_values(['year', 'table_name'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,table_name,dw_total_paid,src_total_paid,total_paid_difference,total_paid_difference_percentage,dw_total_allowed,src_total_allowed,total_allowed_difference,total_allowed_difference_percentage
28,2011,ccaeo,9.789023e+10,97930912848,-40678228.79,0.041538,1.204331e+11,1.204835e+11,-5.035298e+07,0.041792
26,2011,ccaes,4.846168e+10,48530139946,-68455733.17,0.141058,5.301978e+10,5.309506e+10,-7.527157e+07,0.141768
24,2011,mdcro,9.643964e+09,9647446643,-3482660.93,0.036099,3.281372e+10,3.281620e+10,-2.478564e+06,0.007553
29,2011,mdcrs,5.114301e+09,5117238523,-2937407.99,0.057402,2.037308e+10,2.037895e+10,-5.871361e+06,0.028811
27,2012,ccaeo,1.023702e+11,102415611581,-45449297.63,0.044377,1.277247e+11,1.277824e+11,-5.765025e+07,0.045116
25,2012,ccaes,4.996398e+10,50013773704,-49795535.15,0.099564,5.467110e+10,5.472552e+10,-5.441753e+07,0.099437
30,2012,mdcro,8.674936e+09,8678058239,-3122601.15,0.035983,3.213751e+10,3.214170e+10,-4.194030e+06,0.013049
31,2012,mdcrs,4.654027e+09,4655845817,-1818886.49,0.039067,1.907119e+10,1.907359e+10,-2.397466e+06,0.012570
20,2013,ccaeo,8.250650e+10,82548547983,-42047823.63,0.050937,1.028400e+11,1.028898e+11,-4.985058e+07,0.048450
21,2013,ccaes,4.089856e+10,40959390364,-60830867.78,0.148515,4.464948e+10,4.471432e+10,-6.484352e+07,0.145017


### Min

In [25]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.truv_costs_min;

    create table qa_reporting.truv_costs_min
    (
        year int,
        table_name text,
        min_netpay bigint,
        min_pay bigint
    );
    '''
    cursor.execute(query)

In [26]:
for table in tables[:]:
    with connection.cursor() as cursor:
        query = f'''insert into qa_reporting.truv_costs_min
    with truv_claims as (
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.{table}
        group by 1,2
    )
    select year, '{table}' as table_name, min(netpay), min(pay) 
    from truv_claims
    group by 1
        '''
        cursor.execute(query)

In [27]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.dw_truv_costs_min;
select year, table_id_src,
        min(total_charge_amount) as min_total_charge_amount,
        min(total_allowed_amount) as min_total_allowed_amount, 
        min(total_paid_amount) as min_total_paid_amount
into qa_reporting.dw_truv_costs_min
from dw_staging.trum_claim_header
group by 1,2
    '''
    cursor.execute(query)

    query = '''
insert into qa_reporting.dw_truv_costs_min
select year, table_id_src,
        min(total_charge_amount) as min_total_charge_amount,
        min(total_allowed_amount) as min_total_allowed_amount, 
        min(total_paid_amount) as min_total_paid_amount
from dw_staging.truc_claim_header
group by 1,2
    '''
    cursor.execute(query)

In [28]:
query = f'''
select a.year, b.table_name, 
        a.min_total_paid_amount as dw_min_total_paid, b.min_netpay as src_min_paid, 
        a.min_total_paid_amount-b.min_netpay as min_total_paid_difference, 
        100. * abs(a.min_total_paid_amount-b.min_netpay) / b.min_netpay as min_total_paid_difference_percentage,
        a.min_total_allowed_amount as dw_min_total_allowed, b.min_pay as src_min_allowed,
        a.min_total_allowed_amount-b.min_pay as min_total_allowed_difference, 
        100. * abs(a.min_total_allowed_amount-b.min_pay) / b.min_pay as min_total_allowed_difference_percentage
from qa_reporting.dw_truv_costs_min a
full outer join qa_reporting.truv_costs_min b
on a.year =b.year
and a.table_id_src = b.table_name
'''

pd.read_sql(query, con=connection).sort_values(['year', 'table_name'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,table_name,dw_min_total_paid,src_min_paid,min_total_paid_difference,min_total_paid_difference_percentage,dw_min_total_allowed,src_min_allowed,min_total_allowed_difference,min_total_allowed_difference_percentage
23,2011,ccaeo,-365085.75,-365086,0.25,-0.000068,-1598982.22,-1598982.0,-0.22,-0.000014
16,2011,ccaes,-2414961.42,-2414961,-0.42,-0.000017,-8878220.00,-8878220.0,0.00,0.000000
22,2011,mdcro,-58497.00,-58497,0.00,0.000000,-1955955.00,-1955955.0,0.00,0.000000
21,2011,mdcrs,-1985218.00,-1985218,0.00,0.000000,-1985218.00,-1985218.0,0.00,0.000000
18,2012,ccaeo,-431422.47,-431422,-0.47,-0.000109,-915090.00,-915090.0,0.00,0.000000
19,2012,ccaes,-3228180.67,-3228181,0.33,-0.000010,-3228180.67,-3228181.0,0.33,-0.000010
20,2012,mdcro,-107614.94,-107615,0.06,-0.000056,-586311.96,-720835.0,134523.04,-18.662113
17,2012,mdcrs,-859830.64,-859831,0.36,-0.000042,-1340482.00,-1340482.0,0.00,0.000000
32,2013,ccaeo,-628859.17,-628859,-0.17,-0.000027,-763213.50,-763214.0,0.50,-0.000066
34,2013,ccaes,-2500213.33,-2500213,-0.33,-0.000013,-2500213.33,-2500213.0,-0.33,-0.000013


### Max

In [29]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.truv_costs_max;

    create table qa_reporting.truv_costs_max
    (
        year int,
        table_name text,
        max_netpay bigint,
        max_pay bigint
    );
    '''
    cursor.execute(query)

In [30]:
for table in tables:
    with connection.cursor() as cursor:
        query = f'''insert into qa_reporting.truv_costs_max
    with truv_claims as (
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.{table}
        group by 1,2
    )
    select year, '{table}' as table_name, max(netpay), max(pay) 
    from truv_claims
    group by 1
        '''
        cursor.execute(query)

In [31]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.dw_truv_costs_max;
select year, table_id_src,
        max(total_charge_amount) as max_total_charge_amount,
        max(total_allowed_amount) as max_total_allowed_amount, 
        max(total_paid_amount) as max_total_paid_amount
into qa_reporting.dw_truv_costs_max
from dw_staging.trum_claim_header
group by 1,2
    '''
    cursor.execute(query)

    query = '''
insert into qa_reporting.dw_truv_costs_max
select year, table_id_src,
        max(total_charge_amount) as max_total_charge_amount,
        max(total_allowed_amount) as max_total_allowed_amount, 
        max(total_paid_amount) as max_total_paid_amount
from dw_staging.truc_claim_header
group by 1,2
    '''
    cursor.execute(query)

In [32]:
query = '''
select a.year, b.table_name, 
        a.max_total_paid_amount as dw_max_total_paid, b.max_netpay as src_max_paid, 
        a.max_total_paid_amount-b.max_netpay as max_total_paid_difference, 
        100. * abs(a.max_total_paid_amount-b.max_netpay) / b.max_netpay as max_total_paid_difference_percentage,
        a.max_total_allowed_amount as dw_max_total_allowed, b.max_pay as src_max_allowed,
        a.max_total_allowed_amount-b.max_pay as max_total_allowed_difference, 
        100. * abs(a.max_total_allowed_amount-b.max_pay) / b.max_pay as max_total_allowed_difference_percentage
from qa_reporting.dw_truv_costs_max a
full outer join qa_reporting.truv_costs_max b
on a.year =b.year
and a.table_id_src = b.table_name
'''

pd.read_sql(query, con=connection).sort_values(['year', 'table_name'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,table_name,dw_max_total_paid,src_max_paid,max_total_paid_difference,max_total_paid_difference_percentage,dw_max_total_allowed,src_max_allowed,max_total_allowed_difference,max_total_allowed_difference_percentage
29,2011,ccaeo,2462133.31,42340117,-39877983.69,94.184869,2462133.31,50000389.0,-47538255.69,9.507577e+01
25,2011,ccaes,3412327.00,59177118,-55764791.00,94.233705,8878220.00,63381441.0,-54503221.00,8.599240e+01
30,2011,mdcro,500555.00,828083,-327528.00,39.552557,6106155.25,6106155.0,0.25,4.094230e-06
24,2011,mdcrs,1985812.00,1985812,0.00,0.000000,5422836.35,5422836.0,0.35,6.454187e-06
31,2012,ccaeo,1801357.50,38293144,-36491786.50,95.295875,1802211.50,47719911.0,-45917699.50,9.622336e+01
28,2012,ccaes,4314090.00,56421370,-52107280.00,92.353801,4314090.00,60305929.0,-55991839.00,9.284633e+01
26,2012,mdcro,404472.16,611210,-206737.84,33.824355,1068762.00,1068762.0,0.00,0.000000e+00
27,2012,mdcrs,1397399.84,1397400,-0.16,0.000011,3174327.93,3174328.0,-0.07,2.205191e-06
14,2013,ccaeo,1975256.41,33904695,-31929438.59,94.174092,1975256.41,40530165.0,-38554908.59,9.512645e+01
15,2013,ccaes,4922667.00,59838224,-54915557.00,91.773374,4922667.00,63465750.0,-58543083.00,9.224358e+01


### Median

These queries take a while to run. May lose connection when executing, but the query may still be running even if connection is closed.

In [33]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.truv_costs_median;

    create table qa_reporting.truv_costs_median
    (
        year int,
        table_name text,
        median_netpay bigint,
        median_pay bigint
    );
    '''
    cursor.execute(query)

In [34]:
for table in tables:
    print(table)
    with connection.cursor() as cursor:
        query = f'''insert into qa_reporting.truv_costs_median
    with truv_claims as (
        select year, claim_id_derv, sum(netpay) netpay, sum(pay) pay
        from truven.{table}
        group by 1,2
    )
    select year, '{table}' as table_name, median(netpay), median(pay) 
    from truv_claims
    group by 1
        '''
        cursor.execute(query)

ccaes
mdcrs
mdcro
ccaeo


In [35]:
with connection.cursor() as cursor:
    query = '''drop table if exists qa_reporting.dw_truv_costs_median;
select year, table_id_src,
        median(total_charge_amount) as median_total_charge_amount,
        median(total_allowed_amount) as median_total_allowed_amount, 
        median(total_paid_amount) as median_total_paid_amount
into qa_reporting.dw_truv_costs_median
from dw_staging.trum_claim_header
group by 1,2
    '''
    cursor.execute(query)

    query = '''
insert into qa_reporting.dw_truv_costs_median
select year, table_id_src,
        median(total_charge_amount) as median_total_charge_amount,
        median(total_allowed_amount) as median_total_allowed_amount, 
        median(total_paid_amount) as median_total_paid_amount
from dw_staging.truc_claim_header
group by 1,2
    '''
    cursor.execute(query)

In [36]:
query = f'''
select a.year, b.table_name, 
        a.median_total_paid_amount as dw_median_total_paid, b.median_netpay as src_median_paid, 
        a.median_total_paid_amount-b.median_netpay as median_total_paid_difference, 
        100. * abs(a.median_total_paid_amount-b.median_netpay) / b.median_netpay as median_total_paid_difference_percentage,
        a.median_total_allowed_amount as dw_median_total_allowed, b.median_pay as src_median_allowed,
        a.median_total_allowed_amount-b.median_pay as median_total_allowed_difference, 
        100. * abs(a.median_total_allowed_amount-b.median_pay) / b.median_pay as median_total_allowed_difference_percentage
from qa_reporting.dw_truv_costs_median a
full outer join qa_reporting.truv_costs_median b
on a.year =b.year
and a.table_id_src = b.table_name
'''

pd.read_sql(query, con=connection).sort_values(['year', 'table_name'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,table_name,dw_median_total_paid,src_median_paid,median_total_paid_difference,median_total_paid_difference_percentage,dw_median_total_allowed,src_median_allowed,median_total_allowed_difference,median_total_allowed_difference_percentage
39,2011,ccaeo,65.00,65,0.00,0.000000,93.13,93.0,0.13,0.139785
42,2011,ccaes,141.82,142,-0.18,0.126761,172.84,173.0,-0.16,0.092486
26,2011,mdcro,18.00,18,0.00,0.000000,95.79,96.0,-0.21,0.218750
29,2011,mdcrs,24.26,24,0.26,1.083333,106.70,107.0,-0.30,0.280374
33,2012,ccaeo,64.95,65,-0.05,0.076923,95.96,96.0,-0.04,0.041667
40,2012,ccaes,141.32,141,0.32,0.226950,176.24,176.0,0.24,0.136364
3,2012,mdcro,16.20,16,0.20,1.250000,96.03,96.0,0.03,0.031250
11,2012,mdcrs,21.01,21,0.01,0.047619,103.16,103.0,0.16,0.155340
21,2013,ccaeo,65.00,65,0.00,0.000000,96.27,96.0,0.27,0.281250
13,2013,ccaes,147.79,148,-0.21,0.141892,183.09,183.0,0.09,0.049180


## 